In [37]:
!pip install scikit-learn
!pip install seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import seaborn as sns

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\jiaro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\jiaro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [38]:
pierce = pd.read_csv(r"C:\Users\jiaro\.vscode\happiness-regression\pierce_county_house_sales.csv")


In [39]:

pierce['waterfront_type'].fillna('No Waterfront', inplace=True)
pierce['view_quality'].fillna('No View', inplace=True)


pierce['Log_Price'] = np.log(pierce['sale_price'])

pierce['sale_date'] = pd.to_datetime(pierce['sale_date'])

pierce['sale_date_numeric'] = (pd.Timestamp('today') - pierce['sale_date']) // pd.Timedelta(days=1)
pierce = pierce.drop(columns = ['sale_date','stories', 'interior'])
pierce = pierce.dropna()

pierce['garage_square_feet'] = pierce['attached_garage_square_feet'] + pierce['detached_garage_square_feet']

numeric_features = ['sale_date_numeric', 'attic_finished_square_feet', 'garage_square_feet',
                    'house_square_feet', 'basement_square_feet', 'fireplaces', 'year_built',
                    'bedrooms', 'bathrooms']
scaler = MinMaxScaler()
scaled_array = scaler.fit_transform(pierce[numeric_features])
scaled_numeric = pd.DataFrame(scaled_array, columns=numeric_features, index=pierce.index)

pierce_categorical = pd.get_dummies(pierce[['waterfront_type', 'hvac_description', 'exterior', 'roof_cover', 'utility_sewer', 'view_quality']], drop_first=True)



C:\Users\jiaro\AppData\Local\Temp\ipykernel_8088\1826051519.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pierce['waterfront_type'].fillna('No Waterfront', inplace=True)
C:\Users\jiaro\AppData\Local\Temp\ipykernel_8088\1826051519.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [42]:
X_combined = pd.concat([scaled_numeric, pierce_categorical], axis=1)
y = pierce['Log_Price']

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)


model = LinearRegression().fit(X_train, y_train)
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

rmse_log = np.sqrt(mean_squared_error(y_test, y_pred))
r2_log = r2_score(y_test, y_pred)
print(f"Test RMSE (log dollars): {rmse_log}")
print(f"Test R^2 (log dollars): {r2_log}")

# Back-transform predictions to actual dollars
y_test_actual = np.exp(y_test)
y_pred_actual = np.exp(y_pred)

rmse_actual = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))
print(f"Test RMSE (actual dollars): ${rmse_actual:,.2f}")


train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = train_mse**0.5
train_r2 = r2_score(y_train, y_train_pred)

print(f"train mse: {train_mse}")
print(f"train rmse: {train_rmse}")
print(f"train r2: {train_r2}")

Test RMSE (log dollars): 0.31695270664912795
Test R^2 (log dollars): 0.46511932605128414
Test RMSE (actual dollars): $152,773.70
train mse: 0.08880939650056278
train rmse: 0.2980090543935918
train r2: 0.5058953980452867
